In [1]:
# obtener autor-canción de la base de datos
import pymongo
    
myclient = pymongo.MongoClient('localhost', 27017)
mydb = myclient["test"]
cancion_col = mydb["CANCIONES"]

# obtiene las canciones sin letras
canciones_sin_letra = cancion_col.find({'estado': 'SIN LETRA'}).limit(1) # quitar el limite para obtener todas las canciones sin letras

for cancion_pendiente in canciones_sin_letra:
    print(cancion_pendiente)

{'_id': ObjectId('630a2ce69ffbb04a4088c633'), 'cancion': 'Nueva York (Tot*)', 'cantante': 'BAD GYAL', 'estado': 'SIN LETRA'}


In [2]:
# obtiene la cancion y autor
cancion = cancion_pendiente['cancion']
cantante = cancion_pendiente['cantante'] 
print('cancion: ' + cancion)
print('cantante: ' + cantante)

cancion: Nueva York (Tot*)
cantante: BAD GYAL


In [4]:
import urllib.parse
cancion_code_url = urllib.parse.quote(cancion)
cantante_code_url = urllib.parse.quote(cantante)
print('cancion codificado para url: ' + cancion_code_url)
print('cantante codificado para url: ' + cantante_code_url)

cancion codificado para url: Nueva%20York%20%28Tot%2A%29
cantante codificado para url: BAD%20GYAL


In [7]:
# prepara URL de Lyrics para buscar letras por cancion y autor
import urllib.parse
WEB_LYRICS = "https://www.lyrics.com/lyrics/"
#cancion_code_url = 'morad'
#cantante_code_url = 'pelele'
cancion_autor_code_url = cancion_code_url + '%20' + cantante_code_url
print(cancion_autor_code_url)

url_busqueda_lyrics = '%s%s' % (WEB_LYRICS, cancion_autor_code_url) 
print(url_busqueda_lyrics)

Nueva%20York%20%28Tot%2A%29%20BAD%20GYAL
https://www.lyrics.com/lyrics/Nueva%20York%20%28Tot%2A%29%20BAD%20GYAL


In [8]:
import requests
from bs4 import BeautifulSoup

# Obtiene el contenido web del resultado de búsqueda
#url_busqueda_lyrics = 'https://www.lyrics.com/lyrics/nueva%20york%20tot%20bad%20gyal'
page = requests.get(url_busqueda_lyrics)
soup = BeautifulSoup(page.content, "html.parser")
print(soup)

<!DOCTYPE html>

<html lang="en-US">
<head>
<meta content="#830C66" name="theme-color"/>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<title>Lyrics containing the term: Nueva York (Tot*) BAD GYAL</title>
<meta content="A list of lyrics, artists and songs that contain the term &amp;quot;Nueva York (Tot*) BAD GYAL&amp;quot; - from the Lyrics.com website." name="description"/>
<meta content="Nueva York (Tot*) BAD GYAL lyrics, Nueva York (Tot*) BAD GYAL lyric, Nueva York (Tot*) BAD GYAL songs, Nueva York (Tot*) BAD GYAL artists, lyrics for Nueva York (Tot*) BAD GYAL" name="keywords"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>
<base href="https://www.lyrics.com/"/>
<script>
s4Prefix = 'https://static.stands4.com';
version = '1.4.10';
</script>
<link as="style" href="https://static.stands4.com/app_common/css/bootstrap-3.4.1.min.css" rel="preload"/>
<link href="https://static.stands4.com/app_common/css/boo

In [9]:
#obtiene el cuerpo del contenido autor - canción
results = soup.find(id="content-body")
print(results.prettify())

<div class="row" id="content-body">
 <div class="col-xs-12 col-sm-12">
  <div class="row">
   <div class="adunit-middle adunit-middle-2col clearfix">
    <div class="col-xs-12 col-sm-12">
     <div id="div-gpt-ad-Lyrics_IA_300x250_A">
      <script>
       doDisplay('div-gpt-ad-Lyrics_IA_300x250_A');
      </script>
     </div>
     <div id="div-gpt-ad-Lyrics_Mobile_IA_300x250_A">
      <script>
       doDisplay('div-gpt-ad-Lyrics_Mobile_IA_300x250_A');
      </script>
     </div>
    </div>
   </div>
  </div>
  <div class="clearfix row" id="content-filters">
   <div class="col-sm-5 col-xs-12 no-padding-right">
    <div class="col-sm-2 col-xs-1 no-padding">
     <strong>
      Year:
     </strong>
    </div>
    <div class="col-sm-4 col-xs-3 no-padding">
     <div class="input-group">
      <input class="year-filter form-control" data-link="genres.php?genre=" max="2022" min="1900" placeholder="Year" step="1" type="number" value="">
       <input name="page" type="hidden" value="serp"/>

In [10]:
#obtiene los elementos del cuerpo letra del autor - canción y parte de la letra
job_elements = results.find_all("div", class_="sec-lyric clearfix")
print(job_elements)

[]


In [ ]:
# obtiene elementos del autor, cancion y enlace donde está la letra de la canción
url_letra_cancion = ''
for job_element in job_elements:
    cancion_elem = job_element.find("p", class_="lyric-meta-title")
    autor_elem = job_element.find("p", class_="lyric-meta-album-artist")
    
    if autor_elem == None:
        autor_lyrics = ''
    else:
        autor_lyrics = autor_elem.text

    cancion_lyrics = cancion_elem.text
    ref = job_element.find("pre", class_="lyric-body")

    print("cancion lyrics: ", cancion_lyrics)
    print("autor lyrics: ", autor_lyrics)
    print("url letra lyrics: ", ref.attrs['onclick'])
    print("cancion bdd: " + cancion)
    print("autor bdd: " + cantante)
    print('-----------------')

    # valida autor y canción para obtenr el enlace a la letra de la canción
    if cancion_lyrics.casefold() == cancion.casefold() and autor_lyrics.casefold() == cantante.casefold():
        url_letra_cancion =  ref.attrs['onclick']
        print (url_letra_cancion)
       

if url_letra_cancion == '':
    print ('NO ENCONTRO LETRA')
else:
    url_letra_cancion = url_letra_cancion.split('=')[1].replace('\'','')
    print(url_letra_cancion)

    

In [ ]:
import requests
from bs4 import BeautifulSoup
# obtiene contenido de la página web de la letra de la canción
url_letra_cancion = 'https://www.lyrics.com/lyric-lf/6720102/Bad+Gyal/Nueva+York+%28Tot%2A%29'
page_letra_cancion = requests.get(url_letra_cancion)
soup_letra_cancion = BeautifulSoup(page_letra_cancion.content, "html.parser")
print(soup_letra_cancion)

In [ ]:
# obtiene el cuerpo del contenido web de la letra de la canción
results_letra_cancion = soup_letra_cancion.find(id="content-body")
print(results_letra_cancion.prettify())


In [ ]:
# obtiene el contenedor principal donde está la letra de la canción
job_elements_letra = results_letra_cancion.find_all("div", class_="lyric clearfix")
print(job_elements_letra)

In [ ]:
# obtiene la letra de la canción
for job_element in job_elements_letra:
    letra = job_element.find("pre", id="lyric-body-text")
print(letra.text)